# Customer Service Workflow

# Overview
This notebook demonstrates how to build a customer support agent using Toolhouse.ai in just a few lines of code. The agent can access knowledge bases, check business hours, and provide accurate responses 

MCPS/Tools Used: web search, current_time, get_page_content

In [1]:
!pip install toolhouse openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 5.9 MB/s eta 0:00:00


In [9]:
from toolhouse import Toolhouse, Provider
import os
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
ls

sample_data/


In [10]:
# Load environment variables
load_dotenv()

False

In [ ]:
# Set up API keys
TOOLHOUSE_API_KEY = "toolhouse_api_key"
OPENAI_API_KEY = "openai_api_key"

In [14]:
client = OpenAI(api_key=OPENAI_API_KEY)
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider=Provider.OPENAI)

In [19]:
th.set_metadata('timezone', -5)  # ✅ Numeric offset (EST = UTC-5)

In [20]:
MODEL = 'gpt-4o-mini'

In [23]:
def handle_support_request(user_message, context=None):
    """Handle a customer support request using Toolhouse tools"""

    # System prompt for customer support
    system_prompt = """
    You are a professional customer support agent for TechCorp.

    Guidelines:
    - Be helpful, concise, and professional
    - Check business hours before responding to requests
    - If unsure, escalate to human support
    - Operating hours: 6:00 AM - 10:00 PM EST, Monday-Friday

    Always verify the current time before responding to time-sensitive requests.
    When asked about time, use the available tools to get the current time.
    """

    # Prepare messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    # Add context if provided
    if context:
        messages.insert(1, {"role": "system", "content": f"Context: {context}"})

    # Get response with tools
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=1024
    )

    # Process tools and continue conversation
    tool_results = th.run_tools(response, messages)

    # Check if we got tool results and they're not empty
    if tool_results and len(tool_results) > 0:
        # Get final response with tool results
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=tool_results,
            max_tokens=1024
        )
        return final_response.choices[0].message.content
    else:
        # If no tools were used, return the original response
        return response.choices[0].message.content

In [24]:
# Test scenarios
test_queries = [
    "What time is it right now?",
    "What are your business hours?",
    "I'm having trouble with my order #12345",
    "How do I reset my password?",
    "Can someone help me with billing?"
]

# Process each query
for query in test_queries:
    print(f"User: {query}")
    try:
        response = handle_support_request(query)
        print(f"Agent: {response}\n")
    except Exception as e:
        print(f"Error: {e}\n")
    print("-" * 50)

User: What time is it right now?
Agent: I'm trained on data up until October 2023, and now it's May 2025. How can I assist you today?

--------------------------------------------------
User: What are your business hours?
Agent: Our business hours are from 6:00 AM to 10:00 PM EST, Monday through Friday.

--------------------------------------------------
User: I'm having trouble with my order #12345
Agent: I have access to information up until October 2023. How can I assist you today?

--------------------------------------------------
User: How do I reset my password?
Agent: To reset your password, please follow these general steps:

1. Go to the login page of the TechCorp website.
2. Click on the "Forgot Password?" link.
3. Enter the email address associated with your account and submit the form.
4. Check your email for a password reset link. Follow the instructions in the email to create a new password.

If you encounter any issues during this process, please feel free to ask for fu

In [25]:
def enhanced_support_agent(user_message, customer_id=None, order_history=None):
    """Enhanced support with customer context"""

    # Build context string
    context_parts = []
    if customer_id:
        context_parts.append(f"Customer ID: {customer_id}")
    if order_history:
        context_parts.append(f"Recent orders: {order_history}")

    context = " | ".join(context_parts) if context_parts else None

    # Enhanced system prompt
    system_prompt = """
    You are a premium customer support agent with access to:
    - Current time and business hours
    - Knowledge base search
    - Customer order history (when provided)

    Provide personalized, helpful responses. Use available tools to:
    1. Check if within business hours
    2. Search relevant information
    3. Provide accurate, contextual answers
    """

    return handle_support_request(user_message, context)

# Example with customer context
customer_query = "I want to track my recent order"
response = enhanced_support_agent(
    customer_query,
    customer_id="CUST-12345",
    order_history=["#ORD-001", "#ORD-002"]
)
print(f"Enhanced Response: {response}")

Enhanced Response: To track your orders, I found some useful resources:

1. **For Order #ORD-001** and **Order #ORD-002**:
   - You can check your order status through [AfterShip](https://www.aftership.com/brands/offroaddesign.com). Simply enter your tracking number there to get real-time notifications about your parcel's delivery status.

2. In case you are looking for additional information, there are discussions on platforms like [Reddit](https://www.reddit.com/r/usps_complaints/comments/14wa0w6/ord_item_departed_from_origin/) which mention the abbreviation "ORD" referring to the O'Hare International Airport, although it seems unrelated to tracking.

If you have specific tracking numbers for these orders, you might want to use them at the links provided above for the most accurate status updates.
